In [ ]:
#import
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#tf.config.list_physical_devices('GPU')
#print("Executing eagerly:",tf.executing_eagerly())

In [ ]:
#alias options = {'networkA','networkB','networkC','networkD','networkE','networkF'};
alias='networkF'
aux_filename_36_train="_0-2outs_3-6s_50000itr_train"
aux_filename_69_train="_0-2outs_6-9s_50000itr_train"
aux_filename_36="_0out200000_1-2outs_3-6s_50000itr"
aux_filename_69="_0out200000_1-2outs_6-9s_50000itr"

#from google.colab import drive
#drive.mount('/content/drive')
#caminho="/content/drive/MyDrive/Colab_data/"+alias+"/"+alias+"_"

In [ ]:
#import files to train the meta-classifier
caminho="C:/Users/Patricia/Desktop/GDrive/art_ip4_sgl/train/"+alias+"_"

wAllEL1_pi_sigma_v_36_train = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_36_train)
ypredEL1_pi_sigma_v_36_train = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_36_train)
wAllELInf_pp_sig_obs_36_train = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_36_train)
ypredELInf_pp_sig_obs_36_train = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_36_train)
wAllIDS_pp_36_train = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_36_train)
ypredIDS_pp_36_train = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_36_train)
wAllSLRTMO_pp_36_train = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_36_train)
ypredSLRTMO_pp_36_train = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_36_train)
outs_positions_36_train = np.loadtxt(caminho+"outs_positions"+aux_filename_36_train)

wAllEL1_pi_sigma_v_69_train = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_69_train)
ypredEL1_pi_sigma_v_69_train = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_69_train)
wAllELInf_pp_sig_obs_69_train = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_69_train)
ypredELInf_pp_sig_obs_69_train = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_69_train)
wAllIDS_pp_69_train = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_69_train)
ypredIDS_pp_69_train = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_69_train)
wAllSLRTMO_pp_69_train = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_69_train)
ypredSLRTMO_pp_69_train = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_69_train)
outs_positions_69_train = np.loadtxt(caminho+"outs_positions"+aux_filename_69_train)

In [ ]:
#import files to test
caminho="C:/Users/Patricia/Desktop/GDrive/art_ip4_sgl/"+alias+"_"

wAllEL1_pi_sigma_v_36 = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_36)
ypredEL1_pi_sigma_v_36 = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_36)
wAllELInf_pp_sig_obs_36 = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_36)
ypredELInf_pp_sig_obs_36 = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_36)
wAllIDS_pp_36 = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_36)
ypredIDS_pp_36 = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_36)
wAllSLRTMO_pp_36 = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_36)
ypredSLRTMO_pp_36 = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_36)
outs_positions_36 = np.loadtxt(caminho+"outs_positions"+aux_filename_36)

wAllEL1_pi_sigma_v_69 = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_69)
ypredEL1_pi_sigma_v_69 = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_69)
wAllELInf_pp_sig_obs_69 = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_69)
ypredELInf_pp_sig_obs_69 = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_69)
wAllIDS_pp_69 = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_69)
ypredIDS_pp_69 = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_69)
wAllSLRTMO_pp_69 = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_69)
ypredSLRTMO_pp_69 = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_69)
outs_positions_69 = np.loadtxt(caminho+"outs_positions"+aux_filename_69)

In [ ]:
#create sets of trainning, Ev, validation, test
qtd_graphs = 30000

#######train
aux36_train = (ypredIDS_pp_36_train,wAllIDS_pp_36_train,
             ypredEL1_pi_sigma_v_36_train,wAllEL1_pi_sigma_v_36_train,
             ypredELInf_pp_sig_obs_36_train,wAllELInf_pp_sig_obs_36_train,
             ypredSLRTMO_pp_36_train,wAllSLRTMO_pp_36_train)########################## tirar redun sfc
obs_36_train = np.concatenate(aux36_train,axis=1)
label_36_train = outs_positions_36_train

aux69_train = (ypredIDS_pp_69_train,wAllIDS_pp_69_train,
             ypredEL1_pi_sigma_v_69_train,wAllEL1_pi_sigma_v_69_train,
             ypredELInf_pp_sig_obs_69_train,wAllELInf_pp_sig_obs_69_train,
             ypredSLRTMO_pp_69_train,wAllSLRTMO_pp_69_train)########################## tirar redun sfc
obs_69_train = np.concatenate(aux69_train,axis=1)
label_69_train = outs_positions_69_train

#####test
aux36 = (ypredIDS_pp_36,wAllIDS_pp_36,
             ypredEL1_pi_sigma_v_36,wAllEL1_pi_sigma_v_36,
             ypredELInf_pp_sig_obs_36,wAllELInf_pp_sig_obs_36,
             ypredSLRTMO_pp_36,wAllSLRTMO_pp_36)########################## tirar redun sfc
X_test36 = np.concatenate(aux36,axis=1)
y_test36 = outs_positions_36

aux69 = (ypredIDS_pp_69,wAllIDS_pp_69,
             ypredEL1_pi_sigma_v_69,wAllEL1_pi_sigma_v_69,
             ypredELInf_pp_sig_obs_69,wAllELInf_pp_sig_obs_69,
             ypredSLRTMO_pp_69,wAllSLRTMO_pp_69)########################## tirar redun sfc
X_test69 = np.concatenate(aux69,axis=1)
y_test69 = outs_positions_69

######Ev
aux=sum(np.transpose(y_test36))==0        ##### era y_test (usando o "local" da fç..)
X_Ev = X_test36[aux,:]
y_Ev = y_test36[aux,:]


prop2out = 1.0 
X_train36 = np.concatenate((obs_36_train[0:0+qtd_graphs,:],obs_36_train[50000:50000+qtd_graphs,:],
                         obs_36_train[100000:100000+int(qtd_graphs*prop2out),:]),axis=0)
y_train36 = np.concatenate((label_36_train[0:0+qtd_graphs,:],label_36_train[50000:50000+qtd_graphs,:],
                         label_36_train[100000:100000+int(qtd_graphs*prop2out),:]),axis=0)
X_train69 = np.concatenate((obs_69_train[50000:50000+qtd_graphs,:],
                         obs_69_train[100000:100000+int(qtd_graphs*prop2out),:]),axis=0)
y_train69 = np.concatenate((label_69_train[50000:50000+qtd_graphs,:],
                         label_69_train[100000:100000+int(qtd_graphs*prop2out),:]),axis=0)
X_val36 = np.concatenate((obs_36_train[30000:50000,:],obs_36_train[80000:100000,:],obs_36_train[130000:150000,:]),axis=0)
X_val69 = np.concatenate((obs_69_train[30000:50000,:],obs_69_train[80000:100000,:],obs_69_train[130000:150000,:]),axis=0)
y_val36 = np.concatenate((label_36_train[30000:50000,:],label_36_train[80000:100000,:],label_36_train[130000:150000,:]),axis=0)
y_val69 = np.concatenate((label_69_train[30000:50000,:],label_69_train[80000:100000,:],label_69_train[130000:150000,:]),axis=0)


X_train = np.concatenate((X_train36, X_train69),axis=0)
y_train = np.concatenate((y_train36, y_train69),axis=0)

scaler_train = MinMaxScaler() #StandardScaler() #MinMaxScaler()
X_train = scaler_train.fit_transform(X_train)
X_val36 = scaler_train.transform(X_val36)
X_val69 = scaler_train.transform(X_val69)
X_test36 = scaler_train.transform(X_test36)
X_test69 = scaler_train.transform(X_test69)
X_Ev = scaler_train.transform(X_Ev)

X_val = np.concatenate((X_val36, X_val69),axis=0)
y_val = np.concatenate((y_val36, y_val69),axis=0)
X_test = np.concatenate((X_test36, X_test69),axis=0)
y_test = np.concatenate((y_test36, y_test69),axis=0)


print(X_train.shape,y_val.shape,X_test36.shape,y_test36.shape,X_Ev.shape,y_Ev.shape)

(150000, 240) (120000, 30) (300000, 240) (300000, 30) (200000, 240) (200000, 30)


In [ ]:
#function to evaluate the acc of the meta-classifier
def clf_eval(y_pred, y_val):
  acc = 100*accuracy_score(y_pred, y_val)
  print('Accuracy: {}'.format(acc))
  if out3 == 1:
      aux=sum(np.transpose(y_val))==3
      acc3=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
      print('Accuracy 3 out: {}'.format(acc3))
  else:
      aux=sum(np.transpose(y_val))==2
      acc2=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
      print('Accuracy 2 out: {}'.format(acc2))
      aux=sum(np.transpose(y_val))==1
      acc1=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
      print('Accuracy 1 out: {}'.format(acc1))
  aux=sum(np.transpose(y_val))==0
  acc0=100*accuracy_score(y_val[aux,:], y_pred[aux,:])
  print('Accuracy 0 out: {}'.format(acc0))
  if out3 == 0:
    return [acc1, acc2]

In [ ]:
#function to false alarm control
def fp_control(model,FP, X_test, y_test):
  probs = np.array(model.predict(X_Ev))   ### era X_test (usando o "local" da fç..)
  maxs = np.sort(np.amax(probs, axis=1))
  position = int((1-FP)*maxs.shape[0])
  threshold = maxs[position]

  probs = np.array(model.predict(X_test))
  preds = (np.where(probs[:,:] > threshold, 1, 0))
  label = y_test
  acc =clf_eval(preds,label)
  return acc

In [ ]:
#keras MLP
hidden1 = int(X_train.shape[1]*(2/3))
hidden2 = int(X_train.shape[1]*(1/3))
#hidden3 = int(X_train.shape[1]*(1/3))
#hidden4 = int(X_train.shape[1]*(1/3))

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.0001,patience=10, restore_best_weights=True)

# define the keras model
tf.random.set_seed(1234) #1234
#dropout_rate = 0.1
model = Sequential()
model.add(Dense(hidden1, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(Dense(hidden2, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#model.add(Dense(hidden3, activation='relu'))
#model.add(tf.keras.layers.BatchNormalization())
#model.add(Dense(hidden4, activation='relu'))

model.add(Dense(units=y_train.shape[1], activation='sigmoid'))   #sigmoid

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001)) #default:0.001
model.summary()

# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=200, batch_size=32,#int(X_train.shape[0]/40),
          verbose=0,callbacks=[early_stopping],validation_data=(X_val,y_val))#validation_split=0.1)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 160)               38560     
_________________________________________________________________
batch_normalization_10 (Batc (None, 160)               640       
_________________________________________________________________
dense_16 (Dense)             (None, 80)                12880     
_________________________________________________________________
batch_normalization_11 (Batc (None, 80)                320       
_________________________________________________________________
dense_17 (Dense)             (None, 30)                2430      
Total params: 54,830
Trainable params: 54,350
Non-trainable params: 480
_________________________________________________________________


In [ ]:
out3 = 0
print("Cj de Validacao:")
acc36 = fp_control(model, 0.05, X_val36, y_val36)
acc69 = fp_control(model, 0.05, X_val69, y_val69)
print("MSR:", (acc36[0] + acc36[1] + acc69[0] + acc69[1])/4)
print("Cj de Teste:")
acc36 = fp_control(model, 0.05, X_test36, y_test36)
acc69 = fp_control(model, 0.05, X_test69, y_test69)
print("MSR:", (acc36[0] + acc36[1] + acc69[0] + acc69[1])/4)

Cj de Validacao:
Accuracy: 61.431666666666665
Accuracy 2 out: 32.405
Accuracy 1 out: 57.145
Accuracy 0 out: 94.745
Accuracy: 90.95
Accuracy 2 out: 85.34
Accuracy 1 out: 92.765
Accuracy 0 out: 94.745
MSR: 66.91375
Cj de Teste:
Accuracy: 77.947
Accuracy 2 out: 31.669999999999998
Accuracy 1 out: 56.010000000000005
Accuracy 0 out: 95.0005
Accuracy: 92.865
Accuracy 2 out: 84.77799999999999
Accuracy 1 out: 92.41
Accuracy 0 out: 95.0005
MSR: 66.217


In [ ]:
aux_filename_915="_0-2outs_9-15s_10000itr"
aux_filename_36_3out="_0-3outs_3-6s_10000itr"
aux_filename_69_3out="_0-3outs_6-9s_10000itr"

In [ ]:
caminho=caminho="C:/Users/Patricia/Desktop/GDrive/art_ip4_sgl/9-15/"+alias+"_"

wAllEL1_pi_sigma_v_915 = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_915)
ypredEL1_pi_sigma_v_915 = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_915)
wAllELInf_pp_sig_obs_915 = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_915)
ypredELInf_pp_sig_obs_915 = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_915)
wAllIDS_pp_915 = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_915)
ypredIDS_pp_915 = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_915)
wAllSLRTMO_pp_915 = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_915)
ypredSLRTMO_pp_915 = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_915)
outs_positions_915 = np.loadtxt(caminho+"outs_positions"+aux_filename_915)

In [ ]:
caminho=caminho="C:/Users/Patricia/Desktop/GDrive/art_ip4_sgl/3outs/"+alias+"_"

wAllEL1_pi_sigma_v_36_3out = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_36_3out)
ypredEL1_pi_sigma_v_36_3out = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_36_3out)
wAllELInf_pp_sig_obs_36_3out = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_36_3out)
ypredELInf_pp_sig_obs_36_3out = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_36_3out)
wAllIDS_pp_36_3out = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_36_3out)
ypredIDS_pp_36_3out = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_36_3out)
wAllSLRTMO_pp_36_3out = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_36_3out)
ypredSLRTMO_pp_36_3out = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_36_3out)
outs_positions_36_3out = np.loadtxt(caminho+"outs_positions"+aux_filename_36_3out)

wAllEL1_pi_sigma_v_69_3out = np.loadtxt(caminho+"wAllEL1_pi_sigma_v"+aux_filename_69_3out)
ypredEL1_pi_sigma_v_69_3out = np.loadtxt(caminho+"ypredEL1_pi_sigma_v"+aux_filename_69_3out)
wAllELInf_pp_sig_obs_69_3out = np.loadtxt(caminho+"wAllELInf_pp_sig_obs"+aux_filename_69_3out)
ypredELInf_pp_sig_obs_69_3out = np.loadtxt(caminho+"ypredELInf_pp_sig_obs"+aux_filename_69_3out)
wAllIDS_pp_69_3out = np.loadtxt(caminho+"wAllIDS_pp"+aux_filename_69_3out)
ypredIDS_pp_69_3out = np.loadtxt(caminho+"ypredIDS_pp"+aux_filename_69_3out)
wAllSLRTMO_pp_69_3out = np.loadtxt(caminho+"wAllSLRTMO_pp"+aux_filename_69_3out)
ypredSLRTMO_pp_69_3out = np.loadtxt(caminho+"ypredSLRTMO_pp"+aux_filename_69_3out)
outs_positions_69_3out = np.loadtxt(caminho+"outs_positions"+aux_filename_69_3out)

In [ ]:
aux69_train = (ypredIDS_pp_69_train,wAllIDS_pp_69_train,
             ypredEL1_pi_sigma_v_69_train,wAllEL1_pi_sigma_v_69_train,
             ypredELInf_pp_sig_obs_69_train,wAllELInf_pp_sig_obs_69_train,
             ypredSLRTMO_pp_69_train,wAllSLRTMO_pp_69_train)########################## tirar redun sfc

######################### 9-15s
aux915 = (ypredIDS_pp_915,wAllIDS_pp_915,
             ypredEL1_pi_sigma_v_915,wAllEL1_pi_sigma_v_915,
             ypredELInf_pp_sig_obs_915,wAllELInf_pp_sig_obs_915,
             ypredSLRTMO_pp_915,wAllSLRTMO_pp_915)########################## tirar redun sfc
X_test915 = np.concatenate(aux915,axis=1)
y_test915 = outs_positions_915
X_test915 = scaler_train.transform(X_test915)

######################## 3outs
aux36_3out = (ypredIDS_pp_36_3out,wAllIDS_pp_36_3out,
             ypredEL1_pi_sigma_v_36_3out,wAllEL1_pi_sigma_v_36_3out,
             ypredELInf_pp_sig_obs_36_3out,wAllELInf_pp_sig_obs_36_3out,
             ypredSLRTMO_pp_36_3out,wAllSLRTMO_pp_36_3out)########################## tirar redun sfc
X_test36_3out = np.concatenate(aux36_3out,axis=1)
y_test36_3out = outs_positions_36_3out
X_test36_3out = scaler_train.transform(X_test36_3out)

aux69_3out = (ypredIDS_pp_69_3out,wAllIDS_pp_69_3out,
             ypredEL1_pi_sigma_v_69_3out,wAllEL1_pi_sigma_v_69_3out,
             ypredELInf_pp_sig_obs_69_3out,wAllELInf_pp_sig_obs_69_3out,
             ypredSLRTMO_pp_69_3out,wAllSLRTMO_pp_69_3out)########################## tirar redun sfc
X_test69_3out = np.concatenate(aux69_3out,axis=1)
y_test69_3out = outs_positions_69_3out
X_test69_3out = scaler_train.transform(X_test69_3out)

In [ ]:
#9-15
print("9-15:")
acc = fp_control(model,0.05, X_test915, y_test915)

#3outs
print("3outs:")
out3 = 1
acc = fp_control(model,0.05, X_test36_3out, y_test36_3out)
acc = fp_control(model,0.05, X_test69_3out, y_test69_3out)
out3 = 0

9-15:
Accuracy: 94.43666666666667
Accuracy 2 out: 91.38
Accuracy 1 out: 97.27
Accuracy 0 out: 94.66
3outs:
Accuracy: 52.05
Accuracy 3 out: 9.139999999999999
Accuracy 0 out: 94.96
Accuracy: 75.25500000000001
Accuracy 3 out: 55.55
Accuracy 0 out: 94.96
